## <center> Beautiful Soup Tutorial

Установим необходимые библиотеки:

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

### Beautiful Soup basics

Импортируем саму библиотеку

In [2]:
from bs4 import BeautifulSoup

Рассмотрим работу некоторых функций библиотеки на примере игрушечного html-файла:

In [3]:
f = open('small_website.html', 'r')
contents = f.read()

Создаем экземпляр класса BeautifulSoup. Параметры - наш html и библиотека 'lxml' для парсинга.

In [4]:
soup = BeautifulSoup(contents, 'lxml')

Используя функцию prettify(), выведем содержание всего файла в красивом виде

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
 </head>
 <body>
  <h1 id="site_title">
   A Very Small Website
  </h1>
  <hr/>
  <div class="article">
   <h2>
    <a href="article_1.html">
     Small Article 1 Headline
    </a>
   </h2>
   <p>
    Bla bla bla (1)
   </p>
  </div>
  <hr/>
  <div class="article">
   <h2>
    <a href="article_2.html">
     Small Article 2 Headline
    </a>
   </h2>
   <p>
    Bla bla bla (2)
   </p>
  </div>
  <hr/>
  <div class="footer">
   <p>
    One more bla
   </p>
  </div>
 </body>
</html>


Мы можем обращаться к тегам последовательно:

In [6]:
print(soup.div.h2)

<h2><a href="article_1.html">Small Article 1 Headline</a></h2>


In [7]:
print(soup.div.h2.prettify())

<h2>
 <a href="article_1.html">
  Small Article 1 Headline
 </a>
</h2>



...либо используя функцию find(), которая вернет первый найденный тег, либо None, если такого не найдет

In [8]:
one_h1_tag = soup.find('h2')
print(one_h1_tag.prettify())

<h2>
 <a href="article_1.html">
  Small Article 1 Headline
 </a>
</h2>



...либо используея find_all(), чтобы найти все теги

In [9]:
all_h1_tags = soup.find_all('h2')

for h1_tag in all_h1_tags:
    print(h1_tag.prettify())
    print()

<h2>
 <a href="article_1.html">
  Small Article 1 Headline
 </a>
</h2>


<h2>
 <a href="article_2.html">
  Small Article 2 Headline
 </a>
</h2>




Также можно передать вторым аргументом find() параметр class_, чтобы специфицировать тег:

In [10]:
footer_tag = soup.find('div', class_='footer').p
print(footer_tag.prettify())

<p>
 One more bla
</p>



Чтобы получить текст из тега, используем get_text().

In [11]:
print(footer_tag.get_text())

One more bla


In [12]:
f.close()

### Beautiful Soup for scraping data from Coursera

Заимпортим requests для получения html страниц и возможности залогиниться на сайте и pandas для формирования датасета

In [13]:
import requests
import pandas as pd

Если для доступа к нужным данным необходимо сначала авторизоваться, это можно сделать, например, с помощью библиотеки HTTPBasicAuth

In [14]:
from requests.auth import HTTPBasicAuth
  
# Введите ваши данные
your_username = ''
your_password = ''

response = requests.get('https://www.coursera.org/?authMode=login',
            auth = HTTPBasicAuth(your_username, your_password))

Соберем данные о бесплатных курсах на coursera.

In [15]:
url = 'https://www.coursera.org/courses?query=free'

response = requests.get(url)
response = response.content.decode('utf-8')

# Маленькая проверка, т.к. с русскоязычной курерой возникали некоторые 
# беды с кодировкой. Если работает, не обращайте внимания :)
assert 'cds' not in response

soup = BeautifulSoup(response, 'lxml')

С помощью функции inspect(правый клик -> "просмотреть код") в браузере определим теги, в которых лежат нужные нам данные. Как можно заметить, данные по всем курсам на странице хранятся в отдельном списке - отделим его для удобства в переменную courses_info.

In [16]:
courses_info = soup.find('ul', class_='ais-InfiniteHits-list')

Определим функцию для скраппинга необходимых нам полей:

In [17]:
def field_scrapper(html_tag, tag_class, field_case, n_entries=20):
    '''Finds all tags' 'html_tag' texts with class='tag_class' and puts them into field_case.'''
    tags =  courses_info.find_all(html_tag, class_=tag_class)

    for i in range(n_entries):
        field = tags[i].get_text()
        field_case.append(field)

In [18]:
names = [] # имя курса
partners = [] # партнер курса
types = [] # тип - курс либо специализация
ratings = [] # рейтинг курса
n_students = [] # число окончивших курс студентов
levels = [] # уровень сложности курса

# названия тегов для нужных данных ищем с помощью все той же inspect
field_scrapper('a', 'color-primary-text card-title headline-1-text', names)
field_scrapper('span', 'partner-name m-b-1s', partners)
field_scrapper('div', '_jen3vs _1d8rgfy3', types)
field_scrapper('span', 'ratings-text', ratings)
field_scrapper('span', 'enrollment-number', n_students)
field_scrapper('span', 'difficulty', levels)

In [19]:
data = pd.DataFrame({'name' : names,
              'partner' : partners,
              'type' : types, 
              'rating' : ratings,
              'number_of_students' : n_students,
              'level' : levels})
data.head(5)

,name,partner,type,rating,number_of_students,level
0,Machine Learning,Stanford University,COURSE,4.9,4.4m,Mixed
1,Indigenous Canada,University of Alberta,COURSE,4.8,390k,Mixed
2,The Science of Well-Being,Yale University,COURSE,4.9,3.6m,Mixed
3,Financial Markets,Yale University,COURSE,4.8,1m,Beginner
4,Introduction to Psychology,Yale University,COURSE,4.8,730k,Beginner
